# frenet_optimal_trajectory
- 论文+代码解释
    - https://zhuanlan.zhihu.com/p/387034160
    - https://www.notion.so/Frenet-Optimal-Trajectory-Generation-for-Dynamic-Street-Scenarios-in-a-Frenet-Frame-96d5bced98c146d2a9036a1d5fc2b21d
    - https://blog.csdn.net/Yong_Qi2015/article/details/129252937
- 坐标系转换
    - https://zhuanlan.zhihu.com/p/304474902
    - https://blog.csdn.net/qq_36458461/article/details/111935614

In [4]:
# Parameter
MAX_SPEED = 50.0 / 3.6    # 最大速度 maximum speed [m/s]
MAX_ACCEL = 2.0           # 最大加速度 maximum acceleration[m/ss]
MAX_CURVATURE = 1.0       # 最大曲率 maximum curvature[1/m]
MAX_ROAD_WIDTH = 7.0      # 最大道路宽度 maximum road width [m]
D_ROAD_W = 1.0            # 道路宽度采样间隔 road width sampling length [m]
DT = 0.2                  # time tick [s]
MAX_T = 5.0               # 最大预测时间 max prediction time [m]
MIN_T = 4.0               # 最小预测时间 min prediction time [m]
TARGET_SPEED = 30.0 / 3.6 # 目标速度(即纵向的速度保持) target speed [m/s]
D_T_S = 5.0 / 3.6         # 目标速度采样间隔 target speed sampling length [m/s]
N_S_SAMPLE = 1            # 目标速度采样数量 sampling number of target speed
ROBOT_RADIUS = 2.0        # robot radius [m]

# cost weights
K_J = 0.1
K_T = 0.1
K_D = 1.0
K_LAT = 1.0
K_LON = 1.0

## 五次多项式

$$\begin{array}{l,l,l,l,l,l,l,l,l,l,l,l}
f(t) & = a_0 & + & a_1\cdot t & + & a_2\cdot t^2 & + & a_3\cdot t^3 & + & a_4\cdot t^4 & + & a_5\cdot t^5 \\
f^{'}(t) &  = & & a_1 & + & 2 a_2\cdot t & + & 3 a_3\cdot t^2 & + & 4 a_4\cdot t^3 & + & 5 a_5\cdot t^4 \\
f^{''}(t) &  = & & & & 2 a_2 & + & 6 a_3\cdot t & + & 12 a_4\cdot t^2 & + & 20 a_5\cdot t^3
\end{array}$$

已知
$$\begin{array}{l,l}
f(0) = s_{start}, & f^{'}(0) = v_{start}, & f^{''}(0) = a_{start} \\
f(T) = s_{end}, & f^{'}(T) = v_{end}, & f^{''}(T) = a_{end} \\
\end{array}$$
六个条件，可建立六个方程，求解$a_0, a_1, a_2, a_3, a_4, a_5$六个参数


$$\begin{array}{l,l,l}
f(0) = a_0 & \implies & a_0 = s_{start} \\
f^{'}(t) = a_1 & \implies & a_1 = v_{start} \\
f^{''}(t) = 2 a_2 & \implies & a_2 = \frac{a_{start}}{2}
\end{array}$$


$$\begin{array}{l,l,l,l,l,l,l} 
T^3\cdot a_3 & + & T^4\cdot a_4 & + & T^5\cdot a_5 & = f(T) - a_0 - a_1\cdot T - a_2\cdot T^2  & = s_{end} - a_0 - a_1\cdot T - a_2\cdot T^2 \\
3T^2\cdot a_3 & + & 4 T^3\cdot a_4 & + & 5 T^4\cdot a_5 & = f^{'}(T) - a_1 - 2 a_2\cdot T & = v_{end} - a_1 - 2 a_2\cdot T \\
6T\cdot a_3 & + & 12 T^2\cdot a_4 & + & 20 T^3\cdot a_5 & = f^{''}(T) - 2 a_2 & = a_{end} - 2 a_2
\end{array}$$




$$\left[ \begin{matrix}
T^3 & T^4 & T^5 \\
3T^2 & 4 T^3 & 5 T^4 \\
6T & 12 T^2 & 20 T^3
\end{matrix} \right] \left(\begin{matrix} a_3 \\ a_4 \\ a_5 \end{matrix}\right) = 
\left(\begin{matrix}
s_{end} - a_0 - a_1\cdot T - a_2\cdot T^2 \\
v_{end} - a_1 - 2 a_2\cdot T \\
a_{end} - 2 a_2
\end{matrix}\right)$$

In [5]:
class QuinticPolynomial:

    def __init__(self, s_start, v_start, a_start, s_end, v_end, a_end, t_end):
        # calc coefficient of quintic polynomial

        self.a0 = s_start
        self.a1 = v_start
        self.a2 = a_start / 2.0

        A = np.array([[    t_end ** 3,      t_end ** 4,      t_end ** 5],
                      [3 * t_end ** 2,  4 * t_end ** 3,  5 * t_end ** 4],
                      [6 * t_end,      12 * t_end ** 2, 20 * t_end ** 3]])
        b = np.array([s_end - self.a0 - self.a1 * t_end - self.a2 * t_end ** 2,
                      v_end - self.a1 - 2 * self.a2 * t_end,
                      a_end - 2 * self.a2])
        x = np.linalg.solve(A, b)

        self.a3 = x[0]
        self.a4 = x[1]
        self.a5 = x[2]

    def calc_point(self, t):
        xt = self.a0 + self.a1 * t + self.a2 * t ** 2 + \
             self.a3 * t ** 3 + self.a4 * t ** 4 + self.a5 * t ** 5

        return xt

    def calc_first_derivative(self, t):
        xt = self.a1 + 2 * self.a2 * t + \
             3 * self.a3 * t ** 2 + 4 * self.a4 * t ** 3 + 5 * self.a5 * t ** 4

        return xt

    def calc_second_derivative(self, t):
        xt = 2 * self.a2 + 6 * self.a3 * t + 12 * self.a4 * t ** 2 + 20 * self.a5 * t ** 3

        return xt

    def calc_third_derivative(self, t):
        xt = 6 * self.a3 + 24 * self.a4 * t + 60 * self.a5 * t ** 2

        return xt

## 四次多项式

$$\begin{array}{l,l,l,l,l,l,l,l,l,l}
f(t) & = a_0 & + & a_1\cdot t & + & a_2\cdot t^2 & + & a_3\cdot t^3 & + & a_4\cdot t^4 \\
f^{'}(t) &  = & & a_1 & + & 2 a_2\cdot t & + & 3 a_3\cdot t^2 & + & 4 a_4\cdot t^3 \\
f^{''}(t) &  = & & & & 2 a_2 & + & 6 a_3\cdot t & + & 12 a_4\cdot t^2
\end{array}$$

已知
$$\begin{array}{l,l}
f(0) = s_{start}, & f^{'}(0) = v_{start}, & f^{''}(0) = a_{start} \\
 & f^{'}(T) = v_{end}, & f^{''}(T) = a_{end} \\
\end{array}$$
五个条件，可建立五个方程，求解$a_0, a_1, a_2, a_3, a_4$五个参数


$$\begin{array}{l,l,l}
f(0) = a_0 & \implies & a_0 = s_{start} \\
f^{'}(t) = a_1 & \implies & a_1 = v_{start} \\
f^{''}(t) = 2 a_2 & \implies & a_2 = \frac{a_{start}}{2}
\end{array}$$


$$\begin{array}{l,l,l,l,l,l,l} 
3T^2\cdot a_3 & + & 4 T^3\cdot a_4 & = f^{'}(T) - a_1 - 2 a_2\cdot T & = v_{end} - a_1 - 2 a_2\cdot T \\
6T\cdot a_3 & + & 12 T^2\cdot a_4 & = f^{''}(T) - 2 a_2 & = a_{end} - 2 a_2
\end{array}$$




$$\left[ \begin{matrix}
3T^2 & 4 T^3 \\
6T & 12 T^2
\end{matrix} \right] \left(\begin{matrix} a_3 \\ a_4 \end{matrix}\right) = 
\left(\begin{matrix}
v_{end} - a_1 - 2 a_2\cdot T \\
a_{end} - 2 a_2
\end{matrix}\right)$$


In [6]:
class QuarticPolynomial:

    def __init__(self, s_start, v_start, a_start, v_end, a_end, t_end):
        # calc coefficient of quartic polynomial

        self.a0 = s_start
        self.a1 = v_start
        self.a2 = a_start / 2.0

        A = np.array([[3 * t_end ** 2,  4 * t_end ** 3],
                      [6 * t_end,      12 * t_end ** 2]])
        b = np.array([v_end - self.a1 - 2 * self.a2 * t_end,
                      a_end - 2 * self.a2])
        x = np.linalg.solve(A, b)

        self.a3 = x[0]
        self.a4 = x[1]

    def calc_point(self, t):
        xt = self.a0 + self.a1 * t + self.a2 * t ** 2 + \
             self.a3 * t ** 3 + self.a4 * t ** 4

        return xt

    def calc_first_derivative(self, t):
        xt = self.a1 + 2 * self.a2 * t + \
             3 * self.a3 * t ** 2 + 4 * self.a4 * t ** 3

        return xt

    def calc_second_derivative(self, t):
        xt = 2 * self.a2 + 6 * self.a3 * t + 12 * self.a4 * t ** 2

        return xt

    def calc_third_derivative(self, t):
        xt = 6 * self.a3 + 24 * self.a4 * t

        return xt

In [ ]:
class FrenetPath:
    def __init__(self):
        self.t = []
        self.d = []
        self.d_d = []
        self.d_dd = []
        self.d_ddd = []
        self.s = []
        self.s_d = []
        self.s_dd = []
        self.s_ddd = []
        self.cd = 0.0
        self.cv = 0.0
        self.cf = 0.0

        self.x = []
        self.y = []
        self.yaw = []
        self.ds = []
        self.c = []

In [ ]:
def calc_frenet_paths(c_speed, c_d, c_d_d, c_d_dd, s0):
    """
    input:
        s0:      开始状态纵向s
        c_speed: 开始状态纵向ds/dt
        c_d:     开始状态横向l [m]
        c_d_d:   开始状态横向dl/dt [m/s]
        c_d_dd:  开始状态横向d(dl/dt)/dt
    """
    
    frenet_paths = []

    for di in np.arange(-MAX_ROAD_WIDTH, MAX_ROAD_WIDTH, D_ROAD_W):          # di 结束状态横向l=di
        # Lateral motion planning
        for Ti in np.arange(MIN_T, MAX_T, DT):                               # Ti 结束时间
            fp = FrenetPath()

            lat_qp = QuinticPolynomial(c_d, c_d_d, c_d_dd, di, 0.0, 0.0, Ti) # 枚举(di, Ti), 生成横向五次多项式轨迹

            fp.t = [t for t in np.arange(0.0, Ti, DT)]                       # 时刻序列
            fp.d = [lat_qp.calc_point(t) for t in fp.t]                      # 横向l序列
            fp.d_d = [lat_qp.calc_first_derivative(t) for t in fp.t]         # dl/dt 序列
            fp.d_dd = [lat_qp.calc_second_derivative(t) for t in fp.t]       # d(dl/dt)/dt 序列
            fp.d_ddd = [lat_qp.calc_third_derivative(t) for t in fp.t]       # d(d(dl/dt)/dt)/dt 序列

            # Longitudinal motion planning (Velocity keeping)
            for tv in np.arange(TARGET_SPEED - D_T_S * N_S_SAMPLE,           # tv 纵向目标速度ds/dt
                                TARGET_SPEED + D_T_S * N_S_SAMPLE, D_T_S):
                tfp = copy.deepcopy(fp)
                lon_qp = QuarticPolynomial(s0, c_speed, 0.0, tv, 0.0, Ti)    # 枚举(tv, Ti), 生成纵向四次多项式轨迹

                tfp.s = [lon_qp.calc_point(t) for t in fp.t]                 # 纵向s序列
                tfp.s_d = [lon_qp.calc_first_derivative(t) for t in fp.t]    # ds/dt 序列
                tfp.s_dd = [lon_qp.calc_second_derivative(t) for t in fp.t]  # d(ds/dt)/dt 序列
                tfp.s_ddd = [lon_qp.calc_third_derivative(t) for t in fp.t]  # d(d(ds/ds)/dt)/dt 序列

                Jp = sum(np.power(tfp.d_ddd, 2))                             # 横向jerk平方累计和
                Js = sum(np.power(tfp.s_ddd, 2))                             # 纵向jerk平方累计和
                ds = (TARGET_SPEED - tfp.s_d[-1]) ** 2                       # square of diff from target speed

                tfp.cd = K_J * Jp + K_T * Ti + K_D * tfp.d[-1] ** 2          # 公式 2
                tfp.cv = K_J * Js + K_T * Ti + K_D * ds
                tfp.cf = K_LAT * tfp.cd + K_LON * tfp.cv

                frenet_paths.append(tfp)                                     # tfp 横向和纵向轨迹组合
                
    return frenet_paths